In [ ]:
!pip install librosa soundfile
#library for getting audio files and beat analysis
#soudfile --> reads audio files


In [21]:
from google.colab import files

uploaded = files.upload()   # choose your audio file
audio_path = list(uploaded.keys())[0]
audio_path


Saving [First Place & Best Mix] OSU Origins @ Legends 2025 (Feat. Karan Takhar, DholiHardo, Dr. Srimix, KEV7N, Lotus, & Pavit) - DJ TK - SoundLoadMate.com.mp3 to [First Place & Best Mix] OSU Origins @ Legends 2025 (Feat. Karan Takhar, DholiHardo, Dr. Srimix, KEV7N, Lotus, & Pavit) - DJ TK - SoundLoadMate.com.mp3


'[First Place & Best Mix] OSU Origins @ Legends 2025 (Feat. Karan Takhar, DholiHardo, Dr. Srimix, KEV7N, Lotus, & Pavit) - DJ TK - SoundLoadMate.com.mp3'

In [16]:
import librosa
import numpy as np
import pandas as pd

def run_count_coach_from_file(path, sr_target=None):
    # 1. Load audio
    y, sr = librosa.load(path, sr=sr_target, mono=True)
    print(f"Loaded: {path}, duration = {len(y)/sr:.2f}s, sr = {sr}")

    # 2. Beat tracking
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr, units='frames')
    tempo_val = float(np.atleast_1d(tempo)[0])  # ensure float
    beat_frames = np.asarray(beat_frames, dtype=int)
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    print(f"Estimated tempo: {tempo_val:.1f} BPM | Beats detected: {len(beat_times)}")

    # 3. Onset strength as intensity proxy
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)

    valid_idx = beat_frames < len(onset_env)
    beat_frames = beat_frames[valid_idx]
    beat_times = beat_times[valid_idx]

    intensities = onset_env[beat_frames]

    # 4. Normalize intensity 0–1
    if intensities.max() != intensities.min():
        intensities_norm = (intensities - intensities.min()) / (intensities.max() - intensities.min())
    else:
        intensities_norm = np.zeros_like(intensities)

    # 5. Approximate beat durations from time diffs
    if len(beat_times) > 1:
        durations = np.diff(
            np.append(
                beat_times,
                beat_times[-1] + (beat_times[-1] - beat_times[-2])
            )
        )
    else:
        durations = np.array([0.0])

    # 6. Build DataFrame
    df = pd.DataFrame({
        "beat_index": np.arange(len(beat_times)),
        "start_time_sec": beat_times,
        "duration_sec": durations,
        "intensity": intensities_norm
    })

    # 7. Add 8-count structure
    df["count"] = (df["beat_index"] % 8) + 1
    df["eight_count_group"] = (df["beat_index"] // 8) + 1

    return df

In [17]:
df = run_count_coach_from_file(audio_path)
df.head(20)


Loaded: DMAK @ ATL Tamasha 2025 v12.mp3, duration = 407.07s, sr = 44100
Estimated tempo: 97.5 BPM | Beats detected: 655


,beat_index,start_time_sec,duration_sec,intensity,count,eight_count_group
0,0,0.046440,0.615329,0.814386,1,1
1,1,0.661769,0.638549,0.187062,2,1
2,2,1.300317,0.592109,0.153212,3,1
3,3,1.892426,0.603719,0.191103,4,1
4,4,2.496145,0.603719,0.275831,5,1
5,5,3.099864,0.615329,0.184627,6,1
6,6,3.715193,0.626939,0.099960,7,1
7,7,4.342132,0.603719,0.097560,8,1
8,8,4.945850,0.626939,0.157391,1,2
9,9,5.572789,0.603719,0.057469,2,2


In [18]:
import soundfile as sf

# reload original audio (to match sr exactly)
y, sr = librosa.load(audio_path, sr=None, mono=True)

# beat times from df
beat_times = df["start_time_sec"].values

# generate click track aligned to beats
clicks = librosa.clicks(times=beat_times, sr=sr, length=len(y))

# mix song + clicks
click_volume = 0.7  # adjust this if clicks are too loud/quiet
y_mix = y + click_volume * clicks

# normalize to avoid clipping
max_val = np.max(np.abs(y_mix))
if max_val > 1:
    y_mix = y_mix / max_val

out_audio_path = "song_with_metronome.wav"
sf.write(out_audio_path, y_mix, sr)

out_audio_path

'song_with_metronome.wav'

In [19]:
def sec_to_srt(t):
    # t in seconds -> "HH:MM:SS,mmm"
    hours = int(t // 3600)
    t -= hours * 3600
    minutes = int(t // 60)
    t -= minutes * 60
    seconds = int(t)
    millis = int((t - seconds) * 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{millis:03d}"

srt_lines = []
for i, row in df.iterrows():
    idx = i + 1
    start = float(row["start_time_sec"])
    end = start + float(row["duration_sec"])

    start_str = sec_to_srt(start)
    end_str = sec_to_srt(end)

    count_str = str(int(row["count"]))  # 1..8

    srt_lines.append(f"{idx}")
    srt_lines.append(f"{start_str} --> {end_str}")
    srt_lines.append(count_str)
    srt_lines.append("")  # blank line

srt_content = "\n".join(srt_lines)

srt_path = "count_coach.srt"
with open(srt_path, "w") as f:
    f.write(srt_content)

srt_path

'count_coach.srt'

In [ ]:
from IPython.display import Audio

Audio(out_audio_path)